<a href="https://colab.research.google.com/github/Joshitha18/Language_Diarization/blob/main/AE_200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp37-none-any.whl size=5888 sha256=186b033e7c0ff366a117d0307e1acc6566fa54fbb1f4a6a83667668bfa825ea2
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [4]:
import python_speech_features
import numpy as np
import soundfile as sf
import pickle
import cv2
import glob

In [5]:
!gdown --id 1SzFGWDqv7_GNDl0x2nKT-cE0PRiPpoK7

Downloading...
From: https://drive.google.com/uc?id=1SzFGWDqv7_GNDl0x2nKT-cE0PRiPpoK7
To: /content/telugu_feat_1000.pkl
148MB [00:01, 145MB/s]


In [6]:
feat_file = open('telugu_feat_1000.pkl', "rb")
tel_feat = pickle.load(feat_file)

In [7]:
len(tel_feat[0][18])

39

Step 3

In [8]:
import numpy

In [9]:
for i in range(0,len(tel_feat)):
  tel_feat[i] = numpy.stack( tel_feat[i], axis=0 )

t = np.array(tel_feat[0:2467]) t.shape _: samples,19,39 samples: Number of 200 ms segments time steps: No of frames in it features : mfcc

In [10]:
#t.shape _: samples,19,39 
#samples: Number of 200 ms segments 
#time steps: No of frames in it 
#features : mfcc

In [11]:
t = np.array(tel_feat)

In [12]:
t.shape

(24678, 19, 39)

In [13]:
sequence = t

In [14]:
window_length = 19

In [15]:
train_size = 80
val_size = 15
test_size = 5
train, val, test= sequence[:int(len(sequence)*train_size/100)], sequence[int(len(sequence)*train_size/100):-int(len(sequence)*test_size/100)], sequence[-int(len(sequence)*test_size/100):]

In [16]:
train.shape,val.shape,test.shape

((19742, 19, 39), (3703, 19, 39), (1233, 19, 39))

In [17]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, LSTM, RepeatVector
from keras.models import Model
from keras.models import model_from_json
from keras import regularizers
import datetime
import time
import requests as req
import json
import pandas as pd
import pickle
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [29]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras import callbacks
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
# define model
n_in = 19
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(19,39)))
model.add(RepeatVector(n_in))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(39)))

model.compile(optimizer='adam', loss='mse')
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", mode ="min", patience = 7)
model_save = ModelCheckpoint('wts_200.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_delta=0.1, mode='min')

In [30]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 200)               192000    
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 19, 200)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 19, 200)           320800    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 19, 39)            7839      
Total params: 520,639
Trainable params: 520,639
Non-trainable params: 0
_________________________________________________________________


In [31]:
# fit model
model.fit(train, train, epochs=30, verbose=1, batch_size=457,validation_data=(val,val), callbacks=[earlystopping, model_save, reduce_lr_loss])

Epoch 1/30
44/44 [==============================] - 35s 708ms/step - loss: 105.5949 - val_loss: 60.4584
Epoch 2/30
44/44 [==============================] - 30s 687ms/step - loss: 54.3901 - val_loss: 49.5075
Epoch 3/30
44/44 [==============================] - 30s 690ms/step - loss: 45.0175 - val_loss: 42.8189
Epoch 4/30
44/44 [==============================] - 30s 693ms/step - loss: 39.3287 - val_loss: 38.7519
Epoch 5/30
44/44 [==============================] - 30s 692ms/step - loss: 35.4496 - val_loss: 34.2176
Epoch 6/30
44/44 [==============================] - 30s 693ms/step - loss: 31.3469 - val_loss: 30.4451
Epoch 7/30
44/44 [==============================] - 31s 697ms/step - loss: 27.9734 - val_loss: 27.9414
Epoch 8/30
44/44 [==============================] - 31s 695ms/step - loss: 25.5536 - val_loss: 25.7826
Epoch 9/30
44/44 [==============================] - 31s 704ms/step - loss: 23.6415 - val_loss: 24.1794
Epoch 10/30
44/44 [==============================] - 31s 701ms/step - lo

In [32]:
pred = model.predict(test, batch_size=457)

In [41]:
test[0][0]

array([ -1.53395896, -50.87020484,   6.86831458,  14.42068943,
       -28.22137308,  16.14602626,   2.57386885, -13.96200468,
         6.53097399,  -9.19877125,   5.16253294,  -3.80997834,
        -2.71652346,  -0.11512832,   2.03727596,  -2.35826502,
         1.54397025,   1.07184026,  -2.1598969 ,  -2.7152663 ,
         1.17963533,   5.17126893,  -1.83401837,  -2.77733645,
         1.49489952,  -0.86824379,  -0.27046028,   2.02959914,
        -0.27835788,   0.06794585,   0.59164359,  -0.68798001,
        -0.99175307,  -0.10791792,  -0.6969164 ,   0.49798605,
        -1.30548172,  -1.03060626,   0.39822172])

In [42]:
pred[0][0]

array([-4.8244448e+00, -4.4993317e+01,  2.2965453e+00,  1.2380829e+01,
       -1.7705736e+01,  1.1711441e+01,  2.1142179e-01, -7.4696522e+00,
        1.5634246e+01, -7.7564349e+00, -2.6994891e+00,  8.6232483e-01,
       -2.3774849e-01,  3.2122850e-01, -4.4488978e-01, -1.0002151e+00,
        1.9645032e+00, -7.3258954e-01, -1.5009106e+00, -1.6847308e+00,
       -6.0216540e-01,  2.2111971e+00,  8.0453485e-01, -6.8291008e-01,
       -2.7625010e+00,  1.2004116e-01, -1.2249289e-01,  1.9232570e+00,
        8.1911236e-03, -1.0771266e+00,  1.6684997e-01, -1.4732271e+00,
       -6.3018006e-01,  1.9186379e-01, -3.6647528e-01,  7.6718032e-01,
        7.7155238e-01, -1.5727686e+00, -5.4984492e-01], dtype=float32)

In [35]:
out = model.layers[0].output
m = Model(inputs = model.input, outputs=out)

In [39]:
train_output = m.predict(train)
val_output = m.predict(val)
test_output = m.predict(test)
output = m.predict(sequence)

In [40]:
len(output[0])

200